# Regression ANN with best parameters
    find the best approach for learning rate (exponential schedueling - or not)

In [3]:
import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# Results for lr = 0.001:

In [ ]:
count = 0

# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# COMPLETE
data = data.loc[data.index > 2018090000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size=0.2, shuffle=False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
    
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
    
def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40,  
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))        
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.1))        
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = 7)

hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80)
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)

C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/80
40/40 [==============================] - 0s 3ms/step - loss: 17405.4434 - mse: 17405.4434 - mae: 113.1076
Epoch 2/80
40/40 [==============================] - 0s 3ms/step - loss: 14586.1201 - mse: 14586.1201 - mae: 99.5965
Epoch 3/80
40/40 [==============================] - 0s 2ms/step - loss: 6413.8477 - mse: 6413.8477 - mae: 44.7643
Epoch 4/80
40/40 [==============================] - 0s 3ms/step - loss: 4982.5356 - mse: 4982.5356 - mae: 30.2355
Epoch 5/80
40/40 [==============================] - 0s 3ms/step - loss: 5022.3843 - mse: 5022.3843 - mae: 31.3917
Epoch 6/80
40/40 [==============================] - 0s 3ms/step - loss: 5037.6724 - mse: 5037.6724 - mae: 31.4848
Epoch 7/80
40/40 [==============================] - 0s 3ms/step - loss: 5036.8940 - mse: 5036.8936 - mae: 30.6263
Epoch 8/80
40/40 [==============================] - 0s 3ms/step - loss: 4922.5034 - mse: 4922.5034 - mae: 29.2518
Epoch 9/80
40/40 [==============================] - 0s 3ms/step - loss: 4837.8628 -

40/40 [==============================] - 0s 2ms/step - loss: 5066.1079 - mse: 5066.1079 - mae: 29.2030
Epoch 73/80
40/40 [==============================] - 0s 2ms/step - loss: 4937.0806 - mse: 4937.0806 - mae: 29.4870
Epoch 74/80
40/40 [==============================] - 0s 3ms/step - loss: 4795.5947 - mse: 4795.5947 - mae: 29.0222
Epoch 75/80
40/40 [==============================] - 0s 2ms/step - loss: 4908.7134 - mse: 4908.7134 - mae: 28.9514
Epoch 76/80
40/40 [==============================] - 0s 3ms/step - loss: 4819.6855 - mse: 4819.6855 - mae: 29.2689
Epoch 77/80
40/40 [==============================] - 0s 3ms/step - loss: 5033.4722 - mse: 5033.4722 - mae: 29.9331
Epoch 78/80
40/40 [==============================] - 0s 3ms/step - loss: 4855.7129 - mse: 4855.7129 - mae: 28.8137
Epoch 79/80
40/40 [==============================] - 0s 2ms/step - loss: 4819.1343 - mse: 4819.1343 - mae: 28.4469
Epoch 80/80
40/40 [==============================] - 0s 1ms/step - loss: 4653.2676 - mse: 46

79/79 [==============================] - 0s 3ms/step - loss: 3402.1643 - mse: 3402.1643 - mae: 30.5494
Epoch 64/80
79/79 [==============================] - 0s 2ms/step - loss: 3498.1089 - mse: 3498.1089 - mae: 30.6818
Epoch 65/80
79/79 [==============================] - 0s 3ms/step - loss: 3471.5337 - mse: 3471.5337 - mae: 31.0931
Epoch 66/80
79/79 [==============================] - 0s 3ms/step - loss: 3493.7056 - mse: 3493.7056 - mae: 30.3355
Epoch 67/80
79/79 [==============================] - 0s 2ms/step - loss: 3397.8958 - mse: 3397.8958 - mae: 30.0990
Epoch 68/80
79/79 [==============================] - 0s 3ms/step - loss: 3581.5837 - mse: 3581.5837 - mae: 30.5266
Epoch 69/80
79/79 [==============================] - 0s 3ms/step - loss: 3403.4346 - mse: 3403.4346 - mae: 29.5355
Epoch 70/80
79/79 [==============================] - 0s 3ms/step - loss: 3374.0320 - mse: 3374.0320 - mae: 29.3623
Epoch 71/80
79/79 [==============================] - 0s 3ms/step - loss: 3438.7505 - mse: 34

118/118 [==============================] - 0s 2ms/step - loss: 2631.8508 - mse: 2631.8508 - mae: 27.1297
Epoch 53/80
 68/118 [================>.............] - ETA: 0s - loss: 3337.3911 - mse: 3337.3911 - mae: 29.4504

In [ ]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results

# Results for exponential schedueling:

In [ ]:
count = 0

# data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)
data = data.loc[data.index > 2018090000, :]
    
# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)
    
# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)
    
X = X.astype('float64')
X = X.round(20)
    
# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
             X, y, test_size=0.2, shuffle=False)
    
# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
    
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
    
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)
    
def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40,  
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))        
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.1))        
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.1))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

def exponential_decay_fn(epoch):
    return 0.001 * 0.1 ** (epoch / 20)

tscv = TimeSeriesSplit(n_splits = 7)
    
hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 15, epochs = 80, callbacks = [lr_scheduler])
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)

In [ ]:
rmse_cv = []
for i in mse_cv:
    rmse_cv.append(i ** 0.5)
    
results = pd.DataFrame({'rmse_cv':rmse_cv,
              
                        'mae_cv': mae_cv,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor}, index = ['lr = 0.001', 'exp schedueling'])

results

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)